# Indexing Data

Before we can perform question-answering (QA) operations we much initialize a document store which will store all of our source text and allow us to ask questions in reference to this source text. The process of initializing the document store and adding data to it is called *indexing*.

## Load Data

Before we begin indexing we must load and prepare our dataset. We are using an NHS dataset from Deepset, these same files can be downloaded from the demo repository. TK link

We load them from the `/data` directory like so:

In [1]:
from pathlib import Path

paths = [str(x) for x in Path('../data').glob('*.txt')]
paths[:5]

['../data/www.nhs.uk_conditions_pre-eclampsia_.txt',
 '../data/www.nhs.uk_conditions_acute-lymphoblastic-leukaemia_.txt',
 '../data/www.nhs.uk_conditions_homeopathy_.txt',
 '../data/www.nhs.uk_conditions_blushing_.txt',
 '../data/www.nhs.uk_conditions_ehlers-danlos-syndromes_.txt']

These are all plaintext files that contain NHS information on a particular condition like *pre-exlampsia*, *acute-lymphoblastic-leukaemia*, etc. Let's see how many of these files we have...

In [2]:
len(paths)

953

We will load them all into a variabled called `nhs_text`.

In [3]:
nhs_text = []
for path in paths:
    with open(path, 'r') as fp:
        text = fp.read()
        nhs_text.append(text)

Let's see what they look like:

In [4]:
nhs_text[0]

'Cookies on the NHS website\nWe\'ve put some small files called cookies on your device to make our site work.\nWe\'d also like to use analytics cookies. These send information about how our site is used to services called Adobe Analytics, Hotjar and Google Analytics. We use this information to improve our site.\nLet us know if this is OK. We\'ll use a cookie to save your choice. You can read more about our cookies before you choose.\nI\'m OK with analytics cookies Do not use analytics cookies\nSearch the NHS website\nSearch\nMy account\nHealth A-Z\nLive Well\nMental health\nCare and support\nPregnancy\nBrowse\nMore\nHome Health A to Z\nOverview\n-\nPre-eclampsia\nContents\nOverview\nSymptoms\nCauses\nDiagnosis\nTreatment\nComplications\nPre-eclampsia is a condition that affects some pregnant women, usually during the second half of pregnancy (from 20 weeks) or soon after their baby is delivered.\nImportant:\nCoronavirus (COVID-19) update\nIf you\'re well, it\'s really important you go 

In [5]:
nhs_text[1]

"Cookies on the NHS website\nWe've put some small files called cookies on your device to make our site work.\nWe'd also like to use analytics cookies. These send information about how our site is used to services called Adobe Analytics, Hotjar and Google Analytics. We use this information to improve our site.\nLet us know if this is OK. We'll use a cookie to save your choice. You can read more about our cookies before you choose.\nI'm OK with analytics cookies Do not use analytics cookies\nSearch the NHS website\nSearch\nMy account\nHealth A-Z\nLive Well\nMental health\nCare and support\nPregnancy\nBrowse\nMore\nHome Health A to Z\nOverview\n-\nAcute lymphoblastic leukaemia\nContents\nOverview\nDiagnosis\nTreatment\nComplications\nAcute lymphoblastic leukaemia is a type of cancer that affects white blood cells. It progresses quickly and aggressively and requires immediate treatment. Both adults and children can be affected.\nAcute lymphoblastic leukaemia is rare, with around 790 people

Clearly there is some data cleanup required to remove the generic text found on each page.

In [6]:
nhs_text[0][:623]

"Cookies on the NHS website\nWe've put some small files called cookies on your device to make our site work.\nWe'd also like to use analytics cookies. These send information about how our site is used to services called Adobe Analytics, Hotjar and Google Analytics. We use this information to improve our site.\nLet us know if this is OK. We'll use a cookie to save your choice. You can read more about our cookies before you choose.\nI'm OK with analytics cookies Do not use analytics cookies\nSearch the NHS website\nSearch\nMy account\nHealth A-Z\nLive Well\nMental health\nCare and support\nPregnancy\nBrowse\nMore\nHome Health A to Z\n"

In [7]:
nhs_text[2][:623]

"Cookies on the NHS website\nWe've put some small files called cookies on your device to make our site work.\nWe'd also like to use analytics cookies. These send information about how our site is used to services called Adobe Analytics, Hotjar and Google Analytics. We use this information to improve our site.\nLet us know if this is OK. We'll use a cookie to save your choice. You can read more about our cookies before you choose.\nI'm OK with analytics cookies Do not use analytics cookies\nSearch the NHS website\nSearch\nHealth A-Z\nLive Well\nMental health\nCare and support\nPregnancy\nBrowse\nMore\nHome Health A to Z\nHomeopathy\n"

In [8]:
nhs_text[5][:623]

"Cookies on the NHS website\nWe've put some small files called cookies on your device to make our site work.\nWe'd also like to use analytics cookies. These send information about how our site is used to services called Adobe Analytics, Hotjar and Google Analytics. We use this information to improve our site.\nLet us know if this is OK. We'll use a cookie to save your choice. You can read more about our cookies before you choose.\nI'm OK with analytics cookies Do not use analytics cookies\nSearch the NHS website\nSearch\nMy account\nHealth A-Z\nLive Well\nMental health\nCare and support\nPregnancy\nBrowse\nMore\nHome Health A to Z\n"

In [9]:
import re

header = re.compile(r'(?s)Cookies.*\nHome Health A to Z\n')

def clean(text):
    text = header.sub('', text)
    text = re.sub(r'\n+', '\n', text)
    text = text.split("Page last reviewed:")[0]
    return text

In [10]:
clean(nhs_text[0])

'Overview\n-\nPre-eclampsia\nContents\nOverview\nSymptoms\nCauses\nDiagnosis\nTreatment\nComplications\nPre-eclampsia is a condition that affects some pregnant women, usually during the second half of pregnancy (from 20 weeks) or soon after their baby is delivered.\nImportant:\nCoronavirus (COVID-19) update\nIf you\'re well, it\'s really important you go to all your appointments and scans for the health of you and your baby.\nIf you\'re pregnant, hospitals and clinics are making sure it\'s safe for you to go to appointments.\nIf you get symptoms of COVID-19, or you\'re unwell with something other than COVID-19, speak to your midwife or maternity team. They will advise you what to do.\nFind out more about pregnancy and COVID-19\nSymptoms of pre-eclampsia\nEarly signs of pre-eclampsia include having high blood pressure (hypertension) and protein in your urine (proteinuria). \nIt\'s unlikely that you\'ll notice these signs, but they should be picked up during your routine antenatal appoin

In [11]:
clean(nhs_text[100])

"Glue ear\nGlue ear is where the empty middle part of the ear canal fills up with fluid. This can cause temporary hearing loss. It usually clears up within 3 months, but see a GP about any hearing problems.\nCheck if it's glue ear\nThe most common symptom of glue ear is temporary hearing loss. It can affect both ears at the same time.\nOther symptoms may include:\nearache or ear pain\nhearing sounds like ringing or buzzing (tinnitus)\nGlue ear is much more common in children, but adults with glue ear have the same symptoms.\nOther conditions that cause ear pain\nNon-urgent advice:\nSee a GP if your child has hearing problems\nYour child may be struggling to hear if they often:\nspeak loudly\nare difficult to understand\nask people to repeat what they say\nask for the TV or music to be turned up loud\nstruggle to hear people far away\nbecome easily distracted when people are talking\nseem tired and irritable because it's harder to listen\nInformation:\nCoronavirus (COVID-19) update: how

This cleaning function seems to work well enough so we can apply it to our data to returning a cleaner set of results.

In [12]:
nhs_text = [clean(text) for text in nhs_text]

Our embedding model expects no more than 128 *tokens* of text, this translates to roughly 400-600 characters, so we will split the text into chunks of ~500 characters with a few extra conditions:

* We split on newline characters `\n`
* We leave an overlap of one sentence between each chunk (to avoid missing relevant relationships between sentences).

In [13]:
chunked = []
chunk = 500

for i, page in enumerate(nhs_text):
    url = paths[i][8:]
    page = page.split("\n")
    context = ""
    for j in range(len(page)):
        if j != 0 and len(context) == 0:
            context += page[j-1] + " "
        context += page[j] + " "
        if len(context) >= chunk:
            chunked.append({
                "text": context,
                "url": url
            })
            context = ""

---

*If you'd like to keep things simpler, we can avoid adding the sentence overlap and cutting on newline characters with:*

```python
chunked = []
chunk = 500

for i, page in enumerate(nhs_text):
    url = paths[i][8:]
    for i in range(0, len(page), chunk):
        i_end = min(i + chunk, len(page))
        chunked.append({
            "text": page[i:i_end],
            "url": url
        })
```

---

In [14]:
chunked[0]

{'text': "Overview - Pre-eclampsia Contents Overview Symptoms Causes Diagnosis Treatment Complications Pre-eclampsia is a condition that affects some pregnant women, usually during the second half of pregnancy (from 20 weeks) or soon after their baby is delivered. Important: Coronavirus (COVID-19) update If you're well, it's really important you go to all your appointments and scans for the health of you and your baby. If you're pregnant, hospitals and clinics are making sure it's safe for you to go to appointments. ",
 'url': 'www.nhs.uk_conditions_pre-eclampsia_.txt'}

In [15]:
chunked[1]

{'text': "If you're pregnant, hospitals and clinics are making sure it's safe for you to go to appointments. If you get symptoms of COVID-19, or you're unwell with something other than COVID-19, speak to your midwife or maternity team. They will advise you what to do. Find out more about pregnancy and COVID-19 Symptoms of pre-eclampsia Early signs of pre-eclampsia include having high blood pressure (hypertension) and protein in your urine (proteinuria).  It's unlikely that you'll notice these signs, but they should be picked up during your routine antenatal appointments. ",
 'url': 'www.nhs.uk_conditions_pre-eclampsia_.txt'}

Here we can see the sentence overlap `"If you're pregnant, hospitals and clinics are making sure it's safe for you to go to appointments."` shared between the two chunks. We also split chunks between sentences rather than mid-word (which would be the case if cutting on every 500 characters without consideration of newlines).

## Indexing the Data

With our data prepared we can begin indexing it. First, we initialize the document store.

In [16]:
from haystack.document_stores import PineconeDocumentStore

API_KEY = "<<YOUR_API_KEY>>"
INDEX_NAME = "haystack-nhs-jul"

document_store = PineconeDocumentStore(
    api_key=API_KEY,
    index=INDEX_NAME,
    similarity="cosine",
    embedding_dim=768,
    metadata_config={"indexed": ["url"]}
)

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.
INFO - haystack.document_stores.pinecone -  Index statistics: name: haystack-nhs-jul, embedding dimensions: 768, record count: 0


And now we create our documents and add them to the document store.

In [17]:
from haystack import Document
from tqdm.auto import tqdm  # progress bar

batch_size = 256

counter = 0
docs = []

for d in tqdm(chunked):
    # create haystack document object with text content and doc metadata
    doc = Document(
        content=d["text"],
        meta={
            "url": d["url"]
        }
    )
    docs.append(doc)
    counter += 1
    if counter % batch_size == 0:
        # writing docs everytime 10k docs are reached
        document_store.write_documents(docs)
        docs.clear()

if len(docs) > 0:
    document_store.write_documents(docs)

  0%|          | 0/8557 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/256 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/256 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/256 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/256 [00:00<?, ?it/s]

...

Writing Documents:   0%|          | 0/256 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/256 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/256 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/256 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/109 [00:00<?, ?it/s]

Although we have added the documents to our document store, they do not include *embeddings*, which are required for us to perform a vector search. To create these embeddings we need to initialize a retriever model.

In [18]:
from haystack.retriever.dense import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model='flax-sentence-embeddings/all_datasets_v3_mpnet-base',
    model_format="sentence_transformers"
)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.retriever.dense -  Init retriever using embeddings of model flax-sentence-embeddings/all_datasets_v3_mpnet-base


We use this retriever model to create the embeddings by calling the `update_embeddings` method.

In [19]:
document_store.update_embeddings(
    retriever,
    batch_size=16
)

INFO - haystack.document_stores.pinecone -  Updating embeddings for 8521 docs...


Updating Embedding:   0%|          | 0/8521 [00:00<?, ? docs/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

We can check that our documents have been added using a simple `DocumentSearchPipeline`.

In [20]:
from haystack.pipelines import DocumentSearchPipeline
from haystack.utils import print_documents

search_pipe = DocumentSearchPipeline(retriever)
result = search_pipe.run(
    query="Who is affected by pre-eclampsia?",
    params={"Retriever": {"top_k": 2}}
)

print_documents(result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: Who is affected by pre-eclampsia?

{   'content': 'Overview - Pre-eclampsia Contents Overview Symptoms Causes '
               'Diagnosis Treatment Complications Pre-eclampsia is a condition '
               'that affects some pregnant women, usually during the second '
               'half of pregnancy (from 20 weeks) or soon after their baby is '
               "delivered. Important: Coronavirus (COVID-19) update If you're "
               "well, it's really important you go to all your appointments "
               "and scans for the health of you and your baby. If you're "
               "pregnant, hospitals and clinics are making sure it's safe for "
               'you to go to appointments. ',
    'name': None}

{   'content': 'The only way to cure pre-eclampsia is to deliver the baby, so '
               "you'll usually be monitored regularly until it's possible for "
               'your baby to be delivered. This will normally be at around 37 '
               'to 38 w

This all looks good and we're returning relevant information for our query.

This is just the indexing step, eg adding the data/documents + their embeddings to our document store. In the following notebook *01_test_pipeline.ipynb* we will initialize and test a full *extractive QA* pipeline.

---